In [1]:
from common import *
import warnings
warnings.filterwarnings('ignore')
import operator

In [2]:
TEST = True; PROD = False

# -=#=-=#=-=#=-=#=-=#=- Features selection logic -=#=-=#=-=#=-=#=-=#=-=#=-

### -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=- Abstract Function datatype -=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-

In [3]:
from types import FunctionType

##### item checker

In [4]:
def item_check(a):
    if not isinstance(a, tuple):
        a = (a,)
    return a

#### Function super useful datatype

In [56]:
class Function(type):
    def __new__(self, *args, **kw):
        noattach = '_noattach_' in kw
        if len(args) == 1:
            if isinstance(args[0], __class__):
                return args[0]
            assert callable(args[0])
            if 'name' in kw:
                name = kw['name']
            else:
                name = args[0].__name__
            parents = ()
            attrs = {'__call__': args[0]}
            noattach = True
        else:
            name, parents, attrs = args
        for k, v in list(dict.items(attrs)):
            if k == '__getitem__':
                assert isinstance(v, FunctionType), "just for easier work on our side. it is a function(self, index)"
                if isinstance(v(self, 0), FunctionType): # if getitem returns a function
                    old_v = v
                    v = lambda self, a: __class__(old_v(self, item_check(a)))
            if not isinstance(v, classmethod) and callable(v) and not isinstance(v, __class__) and not noattach:
                attrs[k] = classmethod(v)
        
        if '__getitem__' not in attrs:
            def nogetitem():
                raise ValueError(f'Function "{name}" has no __getitem__ defined')
            attrs['__getitem__'] = lambda*a,**k: nogetitem()
            
        return super().__new__(self, name, parents, attrs)
    
    def __init__(self, *args, **kw):
        #SEE('init', args, kw)
        if len(args) == 1:
            return
        name, parents, attrs = args
        for k in attrs:
            v = getattr(self, k)
            #SEE('init ->', k, v)
            if k[:1] != '_' and callable(v):
                setattr(self, k, __class__(v, _noattach_=True, name=k))
        
    def __call__(self, *a, **k):
        return self.__call__(*a, **k)
        
    def __getitem__(self, *a, **k):
        return self.__getitem__(*a, **k)
    
    __invert__ = lambda f: __class__(lambda *a, **k: ~f(*a,**k))
    __and__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) & g(*a,**k))
    __or__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) | g(*a,**k))
    __xor__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) ^ g(*a,**k))

In [57]:
# testing
if TEST:
    class What(metaclass=Function):
        def __call__(self, x, y):
            return np.sqrt(x**2 + y**2)
        def hi(self, *s):
            pass
            #SEE('hi, ' + s[0])
        __getitem__ = lambda self, a: lambda arg: (SEE('this is getitem', arg, arg*a), 'answer')[1]
    What.hi('u')
    What.hi#.__call__
    /SEE What[5].__class__
    /SEE What[5](6)

##### item getter

In [58]:
class Itemgetter():
    def __init__(self, func):
        self._func = Function(func)
    def __getitem__(self, a):
        return self._func(item_check(a))

In [59]:
# testing
if TEST:
    guy = Itemgetter(lambda *a: SEE(*a))
    guy[5.7,8]

In [60]:
# testing yo!
if TEST:
    class Yo():
        def __call__(self, x):
            SEE(x, '!')
    yo = Yo()
    yo.__call__ = lambda x: SEE('fuck', x)
    yo(5)

#### Function query language implementation

In [61]:
def scan(s):
    s = s.replace(' ','').replace('\n','').replace('\t','').replace('\r','') + '\0'
    i0, i = 0, 0
    while i0 < len(s)-1 and i < len(s):
        c = s[i]
        if 'a' <= c <= 'z' or 'A' <= c <= 'Z' or '0' <= c <= '9' or c == '_':
            i += 1
            continue
        elif c == '\0':
            yield s[i0:i]
            break
        elif c in '~&|^()':
            if i0 < i:
                yield s[i0:i]
            yield c
            i0 = i = i+1
        elif c in '[{<':
            if i0 < i:
                yield s[i0:i]
            i0 = i
            op = c
            cl = {'[':']','{':'}','<':'>'}[op]
            opened = 1
            while opened > 0:
                i += 1
                opened += 1 if s[i] == op else -1 if s[i] == cl else 0
            i += 1
            if op != '<': # if this is not a comment
                yield s[i0:i]
            i0 = i
        else:
            raise AssertionError('bad syntax in input to scanner')

            
Token = namedtuple('Token', 'type value')

def tokenize(s):
    prevIsField = False
    for w in s:
        a = w[0]
        if 'a' <= a <= 'z' or 'A' <= a <= 'Z' or '0' <= a <= '9' or a == '_':
            yield Token('field', w)
            prevIsField = True
            continue
        elif w in ['&','^','|']:
            yield Token('op2', w)
        elif w in ['~']:
            yield Token('op1', w)
        elif w in ['(']:
            yield Token('_group', w)
        elif w in [')']:
            yield Token('group_', w)
        elif a == '{':
            assert prevIsField, '{} section must have been preceded by a field name'
            yield Token('op2', 'query')
            yield Token('raw', w)
        elif a == '[':
            if not prevIsField:
                yield Token('self', '')
            yield Token('op2', 'getitem')
            yield Token('raw', w)
        else:
            assert False, f'invalid word encountered in tokenizer input: {w}'
        prevIsField = False
        
class TokenInfo(O()):
    self = 'self, the owning Function instance'
    field = 'field name of Function instance'
    op2 = 'binary operator (look in the map)'
    op1 = 'unary operator (look in the map)'
    _group = "open grouping (just '(')"
    group_ = "close grouping (just ')')"
    raw = "raw data to be fed to operator"


def parse(s):
    stack = []
    for t in s:
        if t.type in ['field', 'self', 'raw']:
            yield t
        elif t.type in ['op2', 'op1']:
            while stack and stack[-1].type in ['op2', 'op1'] and parse.order[t.value] >= parse.order[stack[-1].value]:
                yield stack.pop()
            stack.append(t)
        elif t.type == '_group':
            stack.append(t)
        elif t.type == 'group_':
            while True:
                if not stack:
                    raise AssertionError('too many close parentheses')
                x = stack.pop()
                if x.type == '_group':
                    break
                else:
                    assert x.type in ['op2', 'op1'], 'internal error'
                    yield x
    while stack:
        x = stack.pop()
        if x.type == '_group':
            raise AssertionError('too many open parentheses')
        assert x.type in ['op2', 'op1'], 'internal error'
        yield x
parse.order = {'query': 0, 'getitem': 0, '~': 1, '&': 2, '^': 3, '|': 4}

    
def query(self, qs):
    revpol = list(parse(tokenize(scan(qs))))
    stack = []
    BUG.stack = stack
    for t in revpol:
        if t.type == 'self':
            stack.append(self)
        elif t.type == 'field':
            stack.append(getattr(self, t.value))
        elif t.type == 'raw':
            stack.append(t.value)
        elif t.type == 'op1':
            a = stack.pop()
            x = query.opTbl[t.value](a)
            stack.append(x)
        elif t.type == 'op2':
            b, a = stack.pop(), stack.pop()
            x = query.opTbl[t.value](a, b)
            stack.append(x)
    return stack.pop()

class GetItemHelper(metaclass=Singleton):
    def __getitem__(self, a):
        return a

query.opTbl = {
    '~': operator.__invert__,
    '&': operator.__and__,
    '^': operator.__xor__,
    '|': operator.__or__,
    'query': lambda f, q: f.query(q[1:-1]), #TODO can only go one level deep
    'getitem': lambda f, a: f[eval(f'GetItemHelper()[{a[1:-1]}]')]
}

Function.query = query

In [62]:
# testing function query
if TEST:
    qs = 'what2{f_ck} & no_[5,62,6:6] <whatevercomment> |  [9] & ~(ho_10|[3,5:9])'
    scanned = list(scan(qs))
    assert (scanned == ['what2','{f_ck}','&','no_','[5,62,6:6]','|','[9]','&','~','(','ho_10','|','[3,5:9]',')']), (
            'Unit test fail for scanner')    
    tokens = list(tokenize(scanned)) #TODO make unit test
    parsed = list(parse(tokens)) # TODO make unit test
    SEE(qs)
    SEE('-'*99)
    SEE(scanned)
    SEE('-'*99)
    for line in tokens:
        SEE(line)
    SEE('-'*99)
    for line in parsed:
        SEE(line)

In [63]:
def stringify(self, name=None):
    name = self.__name__ if name is None else name
    cond = lambda field: not(field=='query' or field[:1]=='_' and field!='__getitem__')
    if name == '__getitem__':
        yield '[]'
        return
    yield name
    if not any(cond(f) for f in dir(self)):
        return
    yield '{'
    for field in dir(self):
        if not cond(field):
            continue
        value = getattr(self, field)
        yield from stringify(getattr(self, field), field)
        yield ','
    yield '}'
Function.stringify = stringify

## .!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!. The actual Features subsets definitions!!!! .!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.

In [64]:
def time_slice(e):
    start, stop = e.start if e.start is not None else 0, e.stop if e.stop is not None else 0
    assert start >= 0 and stop >= 0
    return max(start, stop), min(start, stop)

In [72]:
def FFF__returns_range(x, e):
    if e == 0:
        ss = 'oo aoo cc acc (oo-aoo) (cc-acc) (cc-acc){0} (oo-aoo){0} (aoo-acc) it af'.split()
        ret = x in ss
        ret |= '(it,af,it{1},aoo,acc)' in x
        ret |= any(x[-len(s):]==s for s in ss)
        ret |= any(s+'Draw' in x or s+'Min' in x or s+'Max' in x for s in ss)
        return ret
    elif e == 1 and '(it{1},af{1},it{2},aoo{1},acc{1})' in x:
        return True
    elif isinstance(e, int):
        suf = '{' + str(e) + '}'
        return x[-len(suf):] == suf
    elif isinstance(e, slice):
        lrg, sml = time_slice(e)
        ret = ('(it,af,it{1},aoo,acc){0/1}' not in x and '(it{1},af{1},it{2},aoo{1},acc{1})' not in x and
            '({}..{})'.format(sml,lrg-1) in x or '{{{}/{}}}'.format(sml,lrg-1) in x)
        suf1, suf2 = 'Since'+str(lrg), str(lrg)
        ret |= x[-len(suf1):] == suf1
        ret |= x[-len(suf2):] == suf2 and x[-len(suf2)-1] not in '0123456789'
        return ret
    return False
    
def FFF__returns_span(x, e):
    if isinstance(e, int):
        assert e > 0
    suf1, suf2 = 'Since'+str(e), str(e)
    if x[-len(suf1):] == suf1 or x[-len(suf2):] == suf2:
        return True
    if e == 1:
        return any(FFF__returns_range(x, r) for r in [0,1,2,3,4,5])
    elif e == 2:
        return (x[-5:] == '{0/1}' or x[-5:] == '{1/2}') and ',' not in x
    elif e == 3:
        return x[-5:] == '{0/2}' and ',' not in x
    elif e == 5:
        return '(0..4)' in x or '(5..9)' in x or '(10..14)' in x or '(15..19)' in x
    elif e == 10:
        return '(0..9)' in x or '(5..14)' in x or '(10..19)' in x
    elif e == 15:
        return '(0..14)' in x or '(5..19)' in x
    elif e == 20:
        return '(0..19)' in x
    elif isinstance(e, slice):
        lrg, sml = time_slice(e)
        for i in [1,2,3,4,5,10,15,20,21,40,60,62,120,125,250]:
            if sml < i <= lrg and FFF__returns_span(x, i):
                return True
    return False
    

class FFF(metaclass=Function):
    # BIG NOTE item_check isn't necessary for the __getitem__ definitions anymore, it's done automatically by metaclass
    def __call__(self):
        raise AssertionError('FFF itself cannot be called')
    #class Basic(metaclass=Function):
    #    pass
    
    class AssetEnc(metaclass=Function):
        __call__ = lambda f, x: 'assetCode' in x or 'assetName' in x
        Ids = lambda f, x: x in ['assetCodeId', 'assetNameId']
        
        class RandMap(metaclass=Function):
            __call__ = lambda f, x: 'randMap32' in x
            squared = Itemgetter(lambda a: lambda x:
                any(x[::-1].find(('randMap32_' + str(e**2))[::-1]) == 0 for e in item_check(a)))
            __getitem__ = (lambda f, a: lambda x:
                any(x[::-1].find(('randMap32_' + str(e))[::-1]) == 0 for e in item_check(a)))
        
        WTFF = lambda f, x: 'WTFF' in x
        InUni = lambda f, x: 'inUniCount' in x
    
    class Time(metaclass=Function):
        __call__ = lambda f, x: x in ['dayOfYear', 'dayOfWeek']
        long = lambda f, x: x == 'dayOfYear'
        short = lambda f, x: x == 'dayOfWeek'
        
    class Volatility(metaclass=Function):
        __call__ = lambda f, x: 'volatility' in x or 'Volatility' in x
        __getitem__ = lambda f, a: lambda x: any('olatility'+str(e) in x for e in item_check(a))
        
    #class Adjustment(metaclass=Function):
    #    __call__ = lambda f, x: 'adjustmentMean' in x
    #    __getitem__ = lambda f, a: lambda x: any('adjustmentMean'+str(e) in x for e in item_check(a))
    
    class Drawdown(metaclass=Function):
        __call__ = lambda f, x: 'drawdown' in x or 'Drawdown' in x
        __getitem__ = lambda f, a: lambda x: (
            any((
                 'rawdown'+str(e) in x if isinstance(e, int) else
                 'rawdown({}-{})'.format(*time_slice(e)) in x if isinstance(e, slice) else
                 1/0
                ) for e in item_check(a))
        )
    
    class Drawup(metaclass=Function):
        __call__ = lambda f, x: 'drawup' in x or 'Drawup' in x
        __getitem__ = lambda f, a: lambda x: any(
            (
                'rawup'+str(e) in x if isinstance(e, int) else
                'rawup({}-{})'.format(*time_slice(e)) in x if isinstance(e, slice) else
                1/0
            ) for e in item_check(a)
        )
    
    class Price(metaclass=Function):
        __call__ = lambda f, x: x in ['open', 'close']
        Open = lambda f, x: x == 'open'
        Close = lambda f, x: x == 'close'
    
    class Return(metaclass=Function):
        pure = lambda f, x: all(b not in x for b in
                                 ['inSince','axSince','rawdown','rawup','ecordedDays','assetCode','olume','olatility'])
        __call__ = lambda f, x: 'oo' in x or 'cc' in x or x[:2] in ['it', 'af']
        class oo(metaclass=Function):
            __call__ = lambda f, x: 'oo' in x and not any(['aoo' in x, '-oo' in x, 'oo-' in x, ',oo' in x, 'oo,' in x])
            __getitem__ = lambda f, a: lambda x: any(e==0 and x=='oo' or x=='oo'+str(e) for e in item_check(a))
        class aoo(metaclass=Function):
            __call__ = lambda f, x: 'aoo' in x and not any(['-aoo' in x, 'aoo-' in x, ',aoo' in x, 'aoo,' in x])
            __getitem__ = lambda f, a: lambda x: any(e==0 and x=='aoo' or x=='aoo'+str(e) for e in item_check(a))
        cc = lambda f, x: 'cc' in x and not any(['acc' in x, '-cc' in x, 'cc-' in x, ',cc' in x, 'cc,' in x])
        acc = lambda f, x: 'acc' in x and not any(['-acc' in x, 'acc-' in x, ',acc' in x, 'acc,' in x])
        class doo(metaclass=Function):
            __call__ = lambda f, x: '(oo-aoo)' in x
            __getitem__ = lambda f, a: lambda x: any(e==0 and x=='(oo-aoo)' or x=='(oo-aoo)'+str(e) for e in item_check(a))
        dcc = lambda f, x: '(cc-acc)' in x
        daoc = lambda f, x: '(aoo-acc)' in x
        it = lambda f, x: x[:3] in ['it', 'it{']
        af = lambda f, x: x[:3] in ['af', 'af{']
        mix = lambda f, x: x[:3] == '(it'
        combo = lambda f, x: '/' in x or '..' in x
        
        __getitem__ = lambda f, a: lambda x: any(FFF__returns_range(x, e) for e in item_check(a))
        span = Itemgetter(lambda a: lambda x: any(FFF__returns_span(x, e) for e in item_check(a)))
    
#     class Volume(metaclass=Function):
#         __call__ = lambda f, x: 'volume' in x or 'Volume' in x
#         class Ratio(metaclass=Function):
#             '''~Back2 ~Back2{1} ~5 ~10 ~10{5} ~(20-5)'''
#             __call__ = lambda f, x: 'olumeRatio' in x
#             __getitem__ = lambda f, a: lambda x: x in ['volumeRatio'+str(e) for e in item_check(a)]
#         #Log = lambda f, x: 'Log' in x
#         __getitem__ = lambda f, a: lambda x: any('olume'+str(e) in x or (e==1 and x=='volume') for e in item_check(a))
#         dd = lambda f, x: 'olume' in x and 'dd' in x
        
    class FracRec(metaclass=Function):
        __call__ = lambda f, x: 'fracRecordedDays' in x
        __getitem__ = lambda f, a: lambda x: any(x == 'fracRecordedDaysSince'+str(e) for e in item_check(a))

    class Since(metaclass=Function):
        __call__ = lambda f, x: 'Since' in x
        __getitem__ = lambda f, a: lambda x: any('Since'+str(e) in x for e in item_check(a))
        Max = lambda f, x: 'max' in x or 'Max' in x
        Min = lambda f, x: 'min' in x or 'Min' in x
        
    class VP(metaclass=Function):
        __call__ = lambda f, x: x == 'vp' or x == 'm_vp'
        dd = lambda f, x: 'vpdd' in x
        __getitem__ = lambda f, a: lambda x: any(e==1 and x[-2:]=='vp' or 'vp'+str(e) in x for e in item_check(a))
        
    class Weight(metaclass=Function):
        __call__ = lambda f, x: 'wVP_' in x
        
    class Market(metaclass=Function):
        __call__ = lambda f, x: 'm_' in x

In [73]:
# testing my operators and query language
#pd.Series(F.columns.map(FFF.FracRec[125]), index=F.columns)
#pd.Series(F.columns.map(FFF.query('Return{allow} & Return{span[10:]}')), index=F.columns)

In [74]:
#pd.to_pickle((F,P), '/big/data/saves/train_5searchbase.32.pkl')
#F,P = pd.read_pickle('/big/data/saves/train_5searchbase.32.pkl')
pd.set_option('display.max_rows', 2000)

In [75]:
import json
#json.dump(list(F.columns), open('feats.json', 'w'))
feats = json.load(open(top_dir/'work/feats.json', 'r'))

In [76]:
feats += ['vp5','vp10']

In [69]:
# totally outdated but still useful to have here for reference
'''FFF{
    AssetEnc{Ids,InUni,RandMap{[],squared{[<1..10>],},},WTFF<x2>,},
    FracRec{[<Since>],},
    Level<open/close/volume>,
    Volume{Log,[]},
    VolumeRatioMean{[<5,10,10{5},Back2,Back2{1},5ByMean20>],},}
    Price,
    
    Return{[],af,allow,cc,cc0,dcc,doc0,doo,it,mix,oo,oo0,span{[],func,},},
    Since{[21,62,125,250],Max,Min},
    Drawdown{[<5,10,20,10:5,20:10>],},
    Drawup{[<5,10,20,10:5,20:10>],},
    
    Global,
    Time,
    Volatility{[<5,10,20>],}<Since>,
    Adjustment{[<1,10>],},
    VW,
    MarketRet
}''';

In [77]:
fffList = [
    FFF.query('''
        <>AssetEnc{InUni} | FracRec{[62,250]} | VP<market&notmarket>
        | ~Market & (
            <>VP[5]
            | Return{pure} & Return{
                (oo | aoo) & ( ~combo & [0,10:,20:,60:,120:] | combo & ~([10:,20:]) )
            }|Return & Drawdown[10,20:10] & Since[21,62,250] | Since{Min & [21]}
            | VP{dd & [5]} & Drawup[10,20:10] & Since[21,62,250]
        )|Market & Weight & (
            <>Return{ pure & oo & [10:,20:] } | ~Since & Volatility[10,20]
            | Return{ oo & span[1] } & Drawdown[10,20:10] & Since[21,62]
        )
        | Time
    '''),
]

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 4))



AttributeError: type object 'FFF' has no attribute 'Volume'

In [354]:
(lambda x: (print(len(x)), x)[1] ) (list(filter(FFF.query('Market'), feats)))

659


['m_oo',
 'm_oo5',
 'm_oo10',
 'm_oo20',
 'm_oo60',
 'm_wVP',
 'wVPm_oo',
 'wVPm_oo5',
 'wVPm_oo10',
 'wVPm_oo20',
 'wVPm_oo60',
 'm_Volatility5',
 'm_Volatility10',
 'm_Volatility20',
 'm_Volatility60',
 'wVPm_Volatility5',
 'wVPm_Volatility10',
 'wVPm_Volatility20',
 'wVPm_Volatility60',
 'm_ooMaxSince21',
 'm_ooDrawdownSince21',
 'm_ooMaxDrawdown5Since21',
 'm_ooMaxDrawdown10Since21',
 'm_ooMaxDrawdown20Since21',
 'm_ooMaxDrawdown(10-5)Since21',
 'm_ooMaxDrawdown(20-10)Since21',
 'm_ooMaxSince{5}21',
 'm_ooMaxSince{10}21',
 'm_ooMaxSince{20}21',
 'm_ooMinSince21',
 'm_ooDrawupSince21',
 'm_ooMaxDrawup5Since21',
 'm_ooMaxDrawup10Since21',
 'm_ooMaxDrawup20Since21',
 'm_ooMaxDrawup(10-5)Since21',
 'm_ooMaxDrawup(20-10)Since21',
 'm_ooMinSince{5}21',
 'm_ooMinSince{10}21',
 'm_ooMinSince{20}21',
 'm_oo10MaxSince21',
 'm_oo10DrawdownSince21',
 'm_oo10MaxDrawdown5Since21',
 'm_oo10MaxDrawdown10Since21',
 'm_oo10MaxDrawdown20Since21',
 'm_oo10MaxDrawdown(10-5)Since21',
 'm_oo10MaxDrawdown